This notebook is used for the capstone project

In [1]:
import pandas as pd
import numpy as np

#!conda install -c conda-forge geocoder --yes
!pip install geocoder
import geocoder

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries loaded')

     |████████████████████████████████| 102kB 12.8MB/s ta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geo

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)
df = df_list[0]
print(df.head())

  Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


In [3]:
#drop rows containing not assigned Borough
df = df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
# check if any neighborhood field is ot assigned
df[df.Neighborhood == 'Not assigned']

,Postal Code,Borough,Neighborhood


In [ ]:
# reset index to be sequential again 
df_new = df.reset_index(drop=True)
df_new.shape

(103, 3)

In [ ]:
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo.head()

column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
df_Toronto = pd.DataFrame(columns=column_names)

for index in df_geo.index:
    postalcode = df_geo.at[index, 'Postal Code']
    df_new_index = df_new.loc[df_new['Postal Code'] == postalcode].index[0]
    borough = df_new.at[df_new_index, 'Borough']
    neighborhood = df_new.at[df_new_index, 'Neighborhood']
    df_Toronto = df_Toronto.append({'PostalCode': postalcode,
                                    'Borough': borough,
                                    'Neighborhood': neighborhood,
                                    'Latitude': df_geo.at[index, 'Latitude'],
                                    'Longitude': df_geo.at[index, 'Longitude']}, ignore_index=True)

df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [ ]:
"""
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")
postalcode = df_geo.at[0, 'Postal Code']
df_new_index = df_new.loc[df_new['Postal Code'] == postalcode].index[0]
df_new_index
for data in df_geo:
    postalcode = df_geo.at[index, 'Postal Code']
    df_new_index = df_new.loc[df_new['Postal Code'] == postalcode].index[0]
    borough = df_new.at[df_new.index, 'Borough']
    neighborhood = df_new.at[df_new.index, 'Neighborhood']
    df_Toronto = df_Toronto.append({'PostalCode': postalcode,
                                    'Borough': borough,
                                    'Neighborhood': neighbourhood,
                                    'Latitude': df_geo.at[index, 'Latitude']
                                    'Longitude': df.geo.at[index, 'Longitude']})
for data in df_geo:
    print(data)
df_geo.head()
"""

'\ndf_geo = pd.read_csv("http://cocl.us/Geospatial_data")\npostalcode = df_geo.at[0, \'Postal Code\']\ndf_new_index = df_new.loc[df_new[\'Postal Code\'] == postalcode].index[0]\ndf_new_index\nfor data in df_geo:\n    postalcode = df_geo.at[index, \'Postal Code\']\n    df_new_index = df_new.loc[df_new[\'Postal Code\'] == postalcode].index[0]\n    borough = df_new.at[df_new.index, \'Borough\']\n    neighborhood = df_new.at[df_new.index, \'Neighborhood\']\n    df_Toronto = df_Toronto.append({\'PostalCode\': postalcode,\n                                    \'Borough\': borough,\n                                    \'Neighborhood\': neighbourhood,\n                                    \'Latitude\': df_geo.at[index, \'Latitude\']\n                                    \'Longitude\': df.geo.at[index, \'Longitude\']})\nfor data in df_geo:\n    print(data)\ndf_geo.head()\n'

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_Toronto['Borough'].unique()),
        df_Toronto.shape[0]
    )
)

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

The dataframe has 10 boroughs and 103 neighborhoods.
The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
